### Code to connect, acquire and store raw data from red pitaya from both CH1 and CH2 simulatenusly

### using config.json for manual device labelling

### data stored in data/[device]/[date]/session_000_CH#.csv

### author: Augusto

# Data Streaming (bigger buffers WAY MORE DATA)
### data streaming test

go to http://rp-f0b826.local/ > Data Stream Control > Start

### data streaming saving into binary files to optimize storage

In [4]:
import raw_data_adquisition as adq

if __name__ == "__main__":
    try:
        # Load configuration
        config = adq.load_config("config.json")
        adq.streaming_to_binary(config)
    except Exception as e:
        print(f"Error: {e}")


Connecting to Red Pitaya at 192.168.8.214:8900
Connection successful!
Flushing socket buffer...
Streaming data for 5 seconds for session 1...
Session 1 data saved to /Volumes/One Touch/_data/ipad_on/2024-12-02/session_001.bin
Sleeping for 0 seconds...
Flushing socket buffer...
Streaming data for 5 seconds for session 2...
Session 2 data saved to /Volumes/One Touch/_data/ipad_on/2024-12-02/session_002.bin
Sleeping for 0 seconds...
Flushing socket buffer...
Streaming data for 5 seconds for session 3...
Session 3 data saved to /Volumes/One Touch/_data/ipad_on/2024-12-02/session_003.bin
Sleeping for 0 seconds...
Flushing socket buffer...
Streaming data for 5 seconds for session 4...
Session 4 data saved to /Volumes/One Touch/_data/ipad_on/2024-12-02/session_004.bin
Sleeping for 0 seconds...
Flushing socket buffer...
Streaming data for 5 seconds for session 5...
Session 5 data saved to /Volumes/One Touch/_data/ipad_on/2024-12-02/session_005.bin
Sleeping for 0 seconds...
Flushing socket buff

#### function that converts it into a csv, just for manual data quality check pursposes

In [19]:
import numpy as np
import pandas as pd
import os

def convert_binary_to_csv(binary_filepath, csv_filepath, input_range=20.0, adc_bits=14):
    """
    Convert a binary file with interleaved CH1 and CH2 data to a CSV file.

    Parameters:
        binary_filepath (str): Path to the binary file.
        csv_filepath (str): Path to save the converted CSV file.
        input_range (float): Input range of the ADC (default: 20.0 for ±20V with 1:20 divider).
        adc_bits (int): Number of ADC bits (default: 14 for Red Pitaya).
    """
    try:
        # Read the binary file
        with open(binary_filepath, 'rb') as f:
            raw_data = f.read()

        # Unpack raw binary data into 16-bit signed integers
        samples = np.frombuffer(raw_data, dtype=np.int16)

        # Split samples into CH1 and CH2
        ch1_samples = samples[::2]  # Every other sample for CH1
        ch2_samples = samples[1::2]  # Every other sample for CH2

        # Convert raw ADC values to voltages
        voltage_ch1 = np.array(ch1_samples) * (1.0 / (2**15))  # Assuming 14-bit ADC
        voltage_ch2 = np.array(ch2_samples) * (1.0 / (2**15))

        # timestamp = datetime.datetime.now().isoformat()

        # Create a DataFrame
        df = pd.DataFrame({
            'voltage_ch1': voltage_ch1,
            'voltage_ch2': voltage_ch2,
        })

        # Save the DataFrame to a CSV file
        df.to_csv(csv_filepath, index=False)
        print(f"Converted {binary_filepath} to {csv_filepath} successfully.")

    except Exception as e:
        print(f"Error converting {binary_filepath} to CSV: {e}")


# Example Usage
binary_file = "/Volumes/One Touch/data_test/empty/2024-11-23/session_008.bin"

csv_file = "/Volumes/One Touch/data_test/empty/2024-11-23/session_0082.csv"
convert_binary_to_csv(binary_file, csv_file)



Converted /Volumes/One Touch/data_test/empty/2024-11-23/session_008.bin to /Volumes/One Touch/data_test/empty/2024-11-23/session_0082.csv successfully.
